Запускать в google colab на T4

In [1]:
!pip install catboost
!pip install implicit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 8.8 MB/s eta 0:00:00


In [ ]:
! unzip "/content/stage2_team_data (1).zip"

In [14]:
! pip list

Package                                  Version
---------------------------------------- -------------------
absl-py                                  1.4.0
accelerate                               1.12.0
access                                   1.1.9
affine                                   2.4.0
aiofiles                                 24.1.0
aiohappyeyeballs                         2.6.1
aiohttp                                  3.13.2
aiosignal                                1.4.0
aiosqlite                                0.21.0
alabaster                                1.0.0
albucore                                 0.0.24
albumentations                           2.0.8
ale-py                                   0.11.2
alembic                                  1.17.2
altair                                   5.5.0
annotated-types                          0.7.0
antlr4-python3-runtime                   4.9.3
anyio                                    4.12.0
anywidget                           

ИМПОРТ БИБЛИОТЕК И НАСТРОЙКИ

In [1]:
import gc
import random
from collections import defaultdict
import warnings

import numpy as np
import pandas as pd
from tqdm.auto import tqdm

from catboost import CatBoostRanker, Pool
from scipy import sparse
import implicit

warnings.filterwarnings('ignore')

# Фиксируем seed для воспроизводимости результатов
SEED = 42
np.random.seed(SEED)
random.seed(SEED)

# Гиперпараметры генерации негативных примеров
VALIDATION_SPLIT = 0.9          # Доля данных для обучения (по времени)
NUM_POPULAR_NEGATIVES = 400     # Количество популярных книг-негативов на пользователя
TOP_K_POPULAR = 7000            # Топ популярных книг для выборки негативов
HARD_NEGATIVES_COUNT = 7        # Негативы от того же автора/жанра
RANDOM_NEGATIVES_COUNT = 35     # Случайные негативы на пользователя

print("Библиотеки загружены, настройки установлены")

Библиотеки загружены, настройки установлены


ЗАГРУЗКА И ПЕРВИЧНАЯ ОБРАБОТКА ДАННЫХ

In [2]:
print("Загрузка данных...")

# Читаем основные таблицы
interactions_df = pd.read_csv('/content/public/train.csv', parse_dates=['timestamp'])
members_df = pd.read_csv('/content/public/users.csv')
items_df = pd.read_csv('/content/public/books.csv')
item_categories = pd.read_csv('/content/public/book_genres.csv')

# Для каждой книги берём первый (основной) жанр
# Это упрощение, но работает достаточно хорошо
primary_category = item_categories.groupby('book_id')['genre_id'].first().reset_index()
items_df = items_df.merge(primary_category, on='book_id', how='left')
items_df = items_df.drop_duplicates(subset=['book_id']).reset_index(drop=True)

# Максимальная дата в данных - понадобится для расчёта временных признаков
latest_timestamp = interactions_df['timestamp'].max()

print(f"Загружено взаимодействий: {len(interactions_df):,}")
print(f"Пользователей: {len(members_df):,}")
print(f"Книг: {len(items_df):,}")
print(f"Последняя дата: {latest_timestamp}")

Загрузка данных...
Загружено взаимодействий: 269,061
Пользователей: 7,289
Книг: 55,784
Последняя дата: 2021-09-06 00:17:11


ГЕНЕРАЦИЯ ОБУЧАЮЩЕЙ ВЫБОРКИ С НЕГАТИВНЫМИ ПРИМЕРАМИ

In [3]:
def build_training_samples(positive_interactions, full_history, items_metadata,
                           num_popular=NUM_POPULAR_NEGATIVES,
                           num_hard=HARD_NEGATIVES_COUNT,
                           num_random=RANDOM_NEGATIVES_COUNT,
                           top_k=TOP_K_POPULAR):
    """
    Создаёт обучающую выборку из позитивных примеров и трёх типов негативов:
    1. Hard negatives - книги того же автора/жанра (сложные для модели)
    2. Popular negatives - популярные книги, которые пользователь не читал
    3. Random negatives - случайные книги для разнообразия

    Релевантность:
    - 2: прочитанная книга (has_read=1)
    - 1: книга в списке "хочу прочитать" (has_read=0)
    - 0: негативный пример
    """

    # Формируем позитивные примеры с метками релевантности
    positive_samples = positive_interactions[['user_id', 'book_id', 'has_read']].copy()
    positive_samples['relevance'] = positive_samples['has_read'].map({1: 2, 0: 1})

    # Строим индекс: какие книги видел каждый пользователь
    member_seen_items = full_history.groupby('user_id')['book_id'].apply(set).to_dict()

    # Топ популярных книг (по количеству взаимодействий)
    popularity_ranking = full_history['book_id'].value_counts().index.tolist()[:top_k]

    # Словари для быстрого поиска книг по автору и жанру
    item_to_writer = items_metadata.set_index('book_id')['author_id'].to_dict()
    item_to_category = items_metadata.set_index('book_id')['genre_id'].to_dict()

    # Инвертированные индексы: автор -> список книг, жанр -> список книг
    writer_items = defaultdict(list)
    category_items = defaultdict(list)
    for _, row in items_metadata.iterrows():
        item_id = row['book_id']
        writer_id = row.get('author_id', -1)
        category_id = row.get('genre_id', -1)
        writer_items[writer_id].append(item_id)
        category_items[category_id].append(item_id)

    all_item_ids = items_metadata['book_id'].unique()
    negative_samples = []
    unique_members = positive_samples['user_id'].unique()

    # Генерируем негативы для каждого пользователя
    for member_id in tqdm(unique_members, desc='Генерация негативных примеров'):
        seen_items = member_seen_items.get(member_id, set())
        member_positive_items = positive_samples.loc[
            positive_samples['user_id'] == member_id, 'book_id'
        ].unique()

        # === 1. Hard negatives: книги того же автора/жанра ===
        hard_count = 0
        for pos_item in member_positive_items:
            writer = item_to_writer.get(pos_item, -1)
            category = item_to_category.get(pos_item, -1)

            # Собираем кандидатов от того же автора и жанра
            candidates = writer_items.get(writer, []) + category_items.get(category, [])
            random.shuffle(candidates)

            for candidate in candidates:
                if candidate not in seen_items:
                    negative_samples.append((member_id, candidate, 0))
                    hard_count += 1
                    break

            if hard_count >= num_hard:
                break

        # === 2. Popular negatives: популярные книги ===
        pop_count = 0
        for popular_item in popularity_ranking:
            if popular_item not in seen_items:
                negative_samples.append((member_id, popular_item, 0))
                pop_count += 1
            if pop_count >= num_popular:
                break

        # === 3. Random negatives: случайные книги ===
        rand_count = 0
        while rand_count < num_random:
            random_item = int(np.random.choice(all_item_ids))
            if random_item not in seen_items:
                negative_samples.append((member_id, random_item, 0))
                rand_count += 1

    # Объединяем позитивы и негативы
    negatives_df = pd.DataFrame(negative_samples, columns=['user_id', 'book_id', 'relevance'])
    combined_dataset = pd.concat([
        positive_samples.drop('has_read', axis=1),
        negatives_df
    ], ignore_index=True)

    # Перемешиваем для лучшего обучения
    combined_dataset = combined_dataset.sample(frac=1, random_state=SEED).reset_index(drop=True)

    return combined_dataset

print("\nРазделение данных по времени...")

# Разделяем данные: 90% на обучение, 10% на валидацию (по времени)
time_threshold = interactions_df['timestamp'].quantile(VALIDATION_SPLIT)
history_data = interactions_df[interactions_df['timestamp'] < time_threshold].copy()
future_data = interactions_df[interactions_df['timestamp'] >= time_threshold].copy()

print(f"История (до {time_threshold}): {len(history_data):,} записей")
print(f"Будущее (после): {len(future_data):,} записей")

# Создаём валидационный набор
print("\nГенерация валидационного набора...")
validation_samples = build_training_samples(future_data, interactions_df, items_df)
del future_data
gc.collect()

# Для train set делаем ещё одно разделение внутри history
inner_time_threshold = history_data['timestamp'].quantile(0.8)
train_history_subset = history_data[history_data['timestamp'] < inner_time_threshold].copy()
train_future_subset = history_data[history_data['timestamp'] >= inner_time_threshold].copy()

print("\nГенерация обучающего набора...")
training_samples = build_training_samples(train_future_subset, history_data, items_df)

# Освобождаем память
del train_future_subset, history_data
gc.collect()

print(f"\nРазмер train: {len(training_samples):,}")
print(f"Размер validation: {len(validation_samples):,}")


Разделение данных по времени...
История (до 2021-03-20 21:26:42): 242,154 записей
Будущее (после): 26,907 записей

Генерация валидационного набора...


Генерация негативных примеров:   0%|          | 0/3031 [00:00<?, ?it/s]


Генерация обучающего набора...


Генерация негативных примеров:   0%|          | 0/3836 [00:00<?, ?it/s]


Размер train: 1,736,246
Размер validation: 1,358,850


ПОСТРОЕНИЕ ALS-ЭМБЕДДИНГОВ

In [4]:
def compute_als_embeddings(interaction_data, num_factors=48, num_iterations=40, reg_weight=0.05):
    """
    Строит латентные представления пользователей и книг
    с помощью алгоритма ALS (Alternating Least Squares).
    """

    df = interaction_data.copy()
    df['weight'] = df['has_read'].fillna(1).astype(float)

    # Создаём маппинги ID -> индекс
    unique_members = list(df['user_id'].unique())
    unique_items = list(df['book_id'].unique())

    member_to_idx = {m: i for i, m in enumerate(unique_members)}
    item_to_idx = {b: i for i, b in enumerate(unique_items)}

    # Строим разреженную матрицу item x user
    row_indices = df['book_id'].map(item_to_idx).values
    col_indices = df['user_id'].map(member_to_idx).values
    weights = df['weight'].values

    interaction_matrix = sparse.csr_matrix(
        (weights, (row_indices, col_indices)),
        shape=(len(unique_items), len(unique_members))
    )

    print(f"  Матрица взаимодействий: {interaction_matrix.shape} (items x users)")
    print(f"  Уникальных пользователей: {len(unique_members)}")
    print(f"  Уникальных книг: {len(unique_items)}")
    print(f"  Обучение ALS с {num_factors} факторами...")

    # Обучаем ALS модель
    als_model = implicit.als.AlternatingLeastSquares(
        factors=num_factors,
        regularization=reg_weight,
        iterations=num_iterations,
        use_gpu=True
    )
    als_model.fit(interaction_matrix)

    # Пробуем разные способы извлечения в зависимости от версии implicit
    try:
        # Для новых версий implicit (>=0.6)
        item_vectors = als_model.item_factors.to_numpy() if hasattr(als_model.item_factors, 'to_numpy') else als_model.item_factors
        member_vectors = als_model.user_factors.to_numpy() if hasattr(als_model.user_factors, 'to_numpy') else als_model.user_factors
    except:
        # Альтернативный способ
        item_vectors = als_model.item_factors
        member_vectors = als_model.user_factors

    # Если это не numpy array, пробуем другие методы
    if not isinstance(item_vectors, np.ndarray):
        try:
            item_vectors = np.asarray(item_vectors)
            member_vectors = np.asarray(member_vectors)
        except:
            # Для GPU версии может потребоваться перенос на CPU
            item_vectors = als_model.item_factors.get() if hasattr(als_model.item_factors, 'get') else als_model.item_factors
            member_vectors = als_model.user_factors.get() if hasattr(als_model.user_factors, 'get') else als_model.user_factors
            item_vectors = np.asarray(item_vectors)
            member_vectors = np.asarray(member_vectors)

    print(f"  item_vectors shape: {item_vectors.shape}")
    print(f"  member_vectors shape: {member_vectors.shape}")

    if item_vectors.shape[0] == len(unique_members) and member_vectors.shape[0] == len(unique_items):
        print("  Меняем местами user и item факторы...")
        member_vectors, item_vectors = item_vectors, member_vectors

    # Формируем DataFrame с эмбеддингами пользователей
    num_dims = member_vectors.shape[1]
    member_embedding_cols = [f'member_factor_{i}' for i in range(num_dims)]
    member_embeddings = pd.DataFrame(
        data=member_vectors,
        columns=member_embedding_cols
    )
    member_embeddings['user_id'] = unique_members

    # Формируем DataFrame с эмбеддингами книг
    item_embedding_cols = [f'item_factor_{i}' for i in range(num_dims)]
    item_embeddings = pd.DataFrame(
        data=item_vectors,
        columns=item_embedding_cols
    )
    item_embeddings['book_id'] = unique_items

    print(f"  Эмбеддинги пользователей: {member_embeddings.shape}")
    print(f"  Эмбеддинги книг: {item_embeddings.shape}")

    return member_embeddings, item_embeddings

print("Построение ALS-эмбеддингов...")
member_als_embeddings, item_als_embeddings = compute_als_embeddings(
    interactions_df,
    num_factors=48,
    num_iterations=40
)
print("ALS-эмбеддинги готовы!")

Построение ALS-эмбеддингов...
  Матрица взаимодействий: (49944, 7289) (items x users)
  Уникальных пользователей: 7289
  Уникальных книг: 49944
  Обучение ALS с 48 факторами...


  0%|          | 0/40 [00:00<?, ?it/s]

  item_vectors shape: (7289, 48)
  member_vectors shape: (49944, 48)
  Меняем местами user и item факторы...
  Эмбеддинги пользователей: (7289, 49)
  Эмбеддинги книг: (49944, 49)
ALS-эмбеддинги готовы!


TARGET ENCODING И ВРЕМЕННЫЕ ПРИЗНАКИ

In [5]:
def compute_target_encodings_and_decay(history, items_metadata, cutoff_date,
                                        smoothing_factor=10, decay_halflife=30.0):
    """
    Вычисляет:
    1. Target encoding для авторов и жанров (сглаженная вероятность прочтения)
    2. Time-decay popularity для книг (недавние взаимодействия весят больше)
    """

    # Объединяем историю с метаданными книг
    enriched_history = history.merge(
        items_metadata[['book_id', 'author_id', 'genre_id']],
        on='book_id',
        how='left'
    )

    # Глобальная вероятность прочтения (prior)
    global_read_rate = enriched_history['has_read'].mean()

    # === Target Encoding для авторов ===
    writer_stats = enriched_history.groupby('author_id')['has_read'].agg(['mean', 'count']).reset_index()
    # Сглаженное среднее: (sum + prior * m) / (count + m)
    writer_stats['writer_target_enc'] = (
        (writer_stats['mean'] * writer_stats['count'] + global_read_rate * smoothing_factor) /
        (writer_stats['count'] + smoothing_factor)
    )

    # === Target Encoding для жанров ===
    category_stats = enriched_history.groupby('genre_id')['has_read'].agg(['mean', 'count']).reset_index()
    category_stats['category_target_enc'] = (
        (category_stats['mean'] * category_stats['count'] + global_read_rate * smoothing_factor) /
        (category_stats['count'] + smoothing_factor)
    )

    # === Time-decay popularity для книг ===
    # Чем недавнее взаимодействие, тем больший вес оно имеет
    time_data = history[['book_id', 'timestamp']].copy()
    time_data['days_since'] = (
        (pd.to_datetime(cutoff_date) - pd.to_datetime(time_data['timestamp']))
        .dt.total_seconds() / (3600 * 24)
    )
    time_data['decay_weight'] = np.exp(-time_data['days_since'] / decay_halflife)

    item_recency_score = (
        time_data.groupby('book_id')['decay_weight']
        .sum()
        .reset_index()
        .rename(columns={'decay_weight': 'item_recency_popularity'})
    )

    return (
        writer_stats[['author_id', 'writer_target_enc']],
        category_stats[['genre_id', 'category_target_enc']],
        item_recency_score
    )

print("Вычисление target encoding и time-decay...")
writer_te, category_te, item_recency = compute_target_encodings_and_decay(
    interactions_df,
    items_df,
    latest_timestamp
)
print("Target encoding готов!")

Вычисление target encoding и time-decay...
Target encoding готов!


ГЕНЕРАЦИЯ ПРИЗНАКОВ

In [6]:
def generate_features(sample_data, history, members_metadata, items_metadata,
                      reference_time, member_embeddings=None, item_embeddings=None,
                      writer_encoding=None, category_encoding=None, recency_data=None):
    """
    Генерирует полный набор признаков для обучения модели ранжирования.

    Группы признаков:
    - Агрегаты пользователя: количество взаимодействий, средний рейтинг, конверсия
    - Агрегаты книги: популярность, конверсия в прочтение
    - Взаимодействия user-author и user-genre: история с конкретным автором/жанром
    - Target encoding: сглаженные вероятности для автора и жанра
    - Временные признаки: давность активности, возраст книги
    - ALS-эмбеддинги: латентные факторы пользователя и книги
    """

    result = sample_data.copy()

    # === Агрегаты по пользователям ===
    member_stats = history.groupby('user_id').agg(
        member_interaction_count=('book_id', 'count'),
        member_read_count=('has_read', 'sum'),
        member_avg_score=('rating', 'mean'),
        member_last_active=('timestamp', 'max')
    ).reset_index()

    member_stats['member_conversion'] = (
        member_stats['member_read_count'] /
        (member_stats['member_interaction_count'] + 1e-6)
    )

    # === Агрегаты по книгам ===
    item_stats = history.groupby('book_id').agg(
        item_interaction_count=('user_id', 'count'),
        item_read_count=('has_read', 'sum')
    ).reset_index()

    item_stats['item_conversion'] = (
        item_stats['item_read_count'] /
        (item_stats['item_interaction_count'] + 1e-6)
    )
    item_stats['item_popularity_rank'] = (
        item_stats['item_interaction_count'].rank(method='dense', ascending=False)
    )

    # === Merge основных данных ===
    result = result.merge(member_stats, on='user_id', how='left')
    result = result.merge(item_stats, on='book_id', how='left')
    result = result.merge(members_metadata, on='user_id', how='left')
    result = result.merge(items_metadata, on='book_id', how='left')

    # === Target Encoding ===
    if writer_encoding is not None:
        result = result.merge(writer_encoding, on='author_id', how='left')
        fallback_value = writer_encoding['writer_target_enc'].mean()
        result['writer_target_enc'] = result['writer_target_enc'].fillna(fallback_value)

    if category_encoding is not None:
        result = result.merge(category_encoding, on='genre_id', how='left')
        fallback_value = category_encoding['category_target_enc'].mean()
        result['category_target_enc'] = result['category_target_enc'].fillna(fallback_value)

    if recency_data is not None:
        result = result.merge(recency_data, on='book_id', how='left')
        result['item_recency_popularity'] = result['item_recency_popularity'].fillna(0)

    # === Взаимодействия пользователь-автор ===
    enriched_history = history.merge(
        items_metadata[['book_id', 'author_id', 'genre_id']],
        on='book_id',
        how='left'
    )

    member_writer_history = enriched_history.groupby(['user_id', 'author_id']).agg(
        member_writer_interactions=('book_id', 'count'),
        member_writer_reads=('has_read', 'sum')
    ).reset_index()

    member_writer_history['member_writer_conversion'] = (
        member_writer_history['member_writer_reads'] /
        (member_writer_history['member_writer_interactions'] + 1e-6)
    )

    result = result.merge(member_writer_history, on=['user_id', 'author_id'], how='left')
    result['member_writer_interactions'] = result['member_writer_interactions'].fillna(0)
    result['member_writer_reads'] = result['member_writer_reads'].fillna(0)
    result['member_writer_conversion'] = result['member_writer_conversion'].fillna(0)

    # === Взаимодействия пользователь-жанр ===
    member_category_history = enriched_history.groupby(['user_id', 'genre_id']).agg(
        member_category_interactions=('book_id', 'count'),
        member_category_reads=('has_read', 'sum')
    ).reset_index()

    member_category_history['member_category_conversion'] = (
        member_category_history['member_category_reads'] /
        (member_category_history['member_category_interactions'] + 1e-6)
    )

    result = result.merge(member_category_history, on=['user_id', 'genre_id'], how='left')
    result['member_category_interactions'] = result['member_category_interactions'].fillna(0)
    result['member_category_reads'] = result['member_category_reads'].fillna(0)
    result['member_category_conversion'] = result['member_category_conversion'].fillna(0)

    # === Временные признаки ===
    result['days_since_active'] = (
        (pd.to_datetime(reference_time) - pd.to_datetime(result['member_last_active']))
        .dt.total_seconds() / (3600 * 24)
    )
    max_inactive = result['days_since_active'].max(skipna=True)
    result['days_since_active'] = result['days_since_active'].fillna(max_inactive + 1)
    result = result.drop(columns=['member_last_active'], errors='ignore')

    # Возраст книги
    result['item_age_years'] = pd.to_datetime(reference_time).year - result['publication_year']
    median_age = result['item_age_years'].median(skipna=True) or 0
    result['item_age_years'] = result['item_age_years'].fillna(median_age)

    # === ALS-эмбеддинги ===
    if member_embeddings is not None:
        result = result.merge(member_embeddings, on='user_id', how='left')
    if item_embeddings is not None:
        result = result.merge(item_embeddings, on='book_id', how='left')

    # === Обработка категориальных колонок ===
    for col in ['author_id', 'genre_id']:
        if col in result.columns:
            result[col] = result[col].fillna(-1).astype(int)

    if 'gender' in result.columns:
        result['gender'] = result['gender'].fillna(-1).astype(int)

    # === Заполнение пропусков в числовых колонках ===
    numeric_columns = result.select_dtypes(include=[np.number]).columns.tolist()
    for col in numeric_columns:
        if col not in ['user_id', 'book_id', 'relevance']:
            result[col] = result[col].fillna(0)

    # Определяем текстовые колонки (они не будут использоваться как признаки)
    text_columns = [
        c for c in result.columns
        if result[c].dtype == object and c not in ('user_id', 'book_id')
    ]

    return result, text_columns

СОЗДАНИЕ ПРИЗНАКОВ ДЛЯ TRAIN И VALIDATION

In [7]:
print("Генерация признаков для обучающей выборки...")
train_features_df, train_text_cols = generate_features(
    training_samples,
    train_history_subset,
    members_df,
    items_df,
    inner_time_threshold,
    member_embeddings=member_als_embeddings,
    item_embeddings=item_als_embeddings,
    writer_encoding=writer_te,
    category_encoding=category_te,
    recency_data=item_recency
)
train_labels = training_samples['relevance']

print("Генерация признаков для валидационной выборки...")
val_history = interactions_df[interactions_df['timestamp'] < time_threshold]
val_features_df, val_text_cols = generate_features(
    validation_samples,
    val_history,
    members_df,
    items_df,
    time_threshold,
    member_embeddings=member_als_embeddings,
    item_embeddings=item_als_embeddings,
    writer_encoding=writer_te,
    category_encoding=category_te,
    recency_data=item_recency
)
val_labels = validation_samples['relevance']

# Определяем список признаков для модели
excluded_columns = {'user_id', 'book_id', 'relevance', 'timestamp',
                    'title', 'author_name', 'genre_name'}
feature_names = [
    col for col in train_features_df.columns
    if col not in excluded_columns
    and train_features_df[col].dtype != object
]

print(f"Количество признаков: {len(feature_names)}")

# Категориальные признаки для CatBoost
categorical_features = [col for col in ['gender'] if col in feature_names]
print(f"Категориальные признаки: {categorical_features}")

Генерация признаков для обучающей выборки...
Генерация признаков для валидационной выборки...
Количество признаков: 123
Категориальные признаки: ['gender']


ПОДГОТОВКА ДАННЫХ ДЛЯ CATBOOST

In [8]:
def prepare_catboost_data(features_df, labels_series):
    """
    Подготавливает данные для CatBoost Ranker:
    - Сортирует по user_id и relevance (позитивы первыми в группе)
    - Это важно для корректной работы ранкера
    """

    temp_df = features_df.copy()
    temp_df['relevance'] = labels_series.values

    # Сортируем: сначала по user_id, потом по relevance (по убыванию)
    sorted_df = temp_df.sort_values(
        by=['user_id', 'relevance'],
        ascending=[True, False]
    ).reset_index(drop=True)

    features_sorted = sorted_df.drop(columns=['relevance'])
    labels_sorted = sorted_df['relevance']

    return features_sorted, labels_sorted

print("Подготовка данных для CatBoost...")

train_sorted, train_labels_sorted = prepare_catboost_data(train_features_df, train_labels)
val_sorted, val_labels_sorted = prepare_catboost_data(val_features_df, val_labels)

# Конвертируем категориальные признаки
for col in categorical_features:
    if col in train_sorted.columns:
        train_sorted[col] = train_sorted[col].astype('category')
        val_sorted[col] = val_sorted[col].astype('category')

# Создаём Pool объекты для CatBoost
train_pool = Pool(
    data=train_sorted[feature_names],
    label=train_labels_sorted,
    group_id=train_sorted['user_id'].astype(str),
    cat_features=categorical_features if categorical_features else None
)

val_pool = Pool(
    data=val_sorted[feature_names],
    label=val_labels_sorted,
    group_id=val_sorted['user_id'].astype(str),
    cat_features=categorical_features if categorical_features else None
)

print("Данные подготовлены!")

Подготовка данных для CatBoost...
Данные подготовлены!


ОБУЧЕНИЕ АНСАМБЛЯ МОДЕЛЕЙ

In [9]:
# ============================================================================
# БЛОК 9: ОБУЧЕНИЕ ДВУХ МОДЕЛЕЙ
# ============================================================================

print("=" * 60)
print("ОБУЧЕНИЕ МОДЕЛЕЙ")
print("=" * 60)

# Общие параметры
common_params = {
    'iterations': 3000,
    'bagging_temperature': 0.9,
    'border_count': 128,
    'loss_function': 'YetiRank',
    'eval_metric': 'NDCG:top=20',
    'task_type': 'GPU',
    'devices': '0',
    'bootstrap_type': 'Poisson',
    'verbose': 200,
    'early_stopping_rounds': 100,
    'random_seed': SEED
}

# Конфигурации моделей
model_configs = [
    {'depth': 8, 'l2_leaf_reg': 7.0, 'learning_rate': 0.04},
    {'depth': 7, 'l2_leaf_reg': 3.0, 'learning_rate': 0.04},
]

trained_models = []

for i, config in enumerate(model_configs):
    print(f"\n--- Модель {i+1}/{len(model_configs)} ---")
    print(f"depth={config['depth']}, l2_reg={config['l2_leaf_reg']}, lr={config['learning_rate']}")

    params = {**common_params, **config}

    model = CatBoostRanker(**params)
    model.fit(train_pool, eval_set=val_pool, use_best_model=True)
    trained_models.append(model)

    gc.collect()

print(f"\nОбучено моделей: {len(trained_models)}")

ОБУЧЕНИЕ МОДЕЛЕЙ

--- Модель 1/2 ---
depth=8, l2_reg=7.0, lr=0.04
Groupwise loss function. OneHotMaxSize set to 10


Default metric period is 5 because NDCG is/are not implemented for GPU
Metric NDCG:type=Base is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
Metric NDCG:top=20;type=Base is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


0:	test: 0.1976756	best: 0.1976756 (0)	total: 607ms	remaining: 30m 20s
200:	test: 0.5168070	best: 0.5168070 (200)	total: 31.2s	remaining: 7m 14s
400:	test: 0.5401348	best: 0.5401348 (400)	total: 1m 2s	remaining: 6m 47s
600:	test: 0.5558512	best: 0.5559613 (596)	total: 1m 34s	remaining: 6m 16s
800:	test: 0.5710153	best: 0.5710153 (800)	total: 2m 6s	remaining: 5m 47s
1000:	test: 0.5805234	best: 0.5805234 (1000)	total: 2m 37s	remaining: 5m 15s
1200:	test: 0.5890586	best: 0.5890721 (1199)	total: 3m 9s	remaining: 4m 44s
1400:	test: 0.5971385	best: 0.5971385 (1400)	total: 3m 40s	remaining: 4m 11s
1600:	test: 0.6049150	best: 0.6050503 (1597)	total: 4m 19s	remaining: 3m 47s
1800:	test: 0.6102036	best: 0.6102516 (1797)	total: 4m 51s	remaining: 3m 14s
2000:	test: 0.6146867	best: 0.6146867 (2000)	total: 5m 22s	remaining: 2m 41s
2200:	test: 0.6181422	best: 0.6181918 (2197)	total: 5m 54s	remaining: 2m 8s
2400:	test: 0.6218692	best: 0.6218878 (2399)	total: 6m 27s	remaining: 1m 36s
2600:	test: 0.6243

Default metric period is 5 because NDCG is/are not implemented for GPU
Metric NDCG:type=Base is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
Metric NDCG:top=20;type=Base is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


0:	test: 0.1981849	best: 0.1981849 (0)	total: 336ms	remaining: 16m 49s
200:	test: 0.5271172	best: 0.5272686 (199)	total: 29.8s	remaining: 6m 55s
400:	test: 0.5403081	best: 0.5403632 (392)	total: 1m	remaining: 6m 31s
600:	test: 0.5492617	best: 0.5492730 (599)	total: 1m 30s	remaining: 6m 2s
800:	test: 0.5562835	best: 0.5562835 (800)	total: 2m	remaining: 5m 32s
1000:	test: 0.5652532	best: 0.5654116 (996)	total: 2m 31s	remaining: 5m 2s
1200:	test: 0.5718376	best: 0.5718970 (1198)	total: 3m 1s	remaining: 4m 32s
1400:	test: 0.5800282	best: 0.5800282 (1400)	total: 3m 32s	remaining: 4m 2s
1600:	test: 0.5860341	best: 0.5860608 (1596)	total: 4m 2s	remaining: 3m 31s
1800:	test: 0.5916207	best: 0.5916207 (1800)	total: 4m 32s	remaining: 3m 1s
2000:	test: 0.5969235	best: 0.5969545 (1998)	total: 5m 3s	remaining: 2m 31s
2200:	test: 0.6029541	best: 0.6029541 (2200)	total: 5m 33s	remaining: 2m 1s
2400:	test: 0.6061871	best: 0.6061871 (2400)	total: 6m 4s	remaining: 1m 30s
2600:	test: 0.6097857	best: 0.60

In [13]:
# ============================================================================
# ОЧИСТКА ПАМЯТИ (без потери важных переменных)
# ============================================================================

import gc
import torch
gc.collect()
if 'torch' in dir():
    torch.cuda.empty_cache()
    torch.cuda.synchronize()
try:
    import cupy as cp
    mempool = cp.get_default_memory_pool()
    mempool.free_all_blocks()
    print(f"CuPy память очищена")
except:
    pass
print("\n--- СОСТОЯНИЕ ПАМЯТИ ---")
import psutil
ram = psutil.virtual_memory()
print(f"RAM: {ram.used / 1e9:.1f} / {ram.total / 1e9:.1f} GB ({ram.percent}%)")
try:
    import subprocess
    result = subprocess.run(['nvidia-smi', '--query-gpu=memory.used,memory.total', '--format=csv,nounits,noheader'],
                          capture_output=True, text=True)
    gpu_used, gpu_total = map(int, result.stdout.strip().split(', '))
    print(f"GPU: {gpu_used / 1024:.1f} / {gpu_total / 1024:.1f} GB ({gpu_used/gpu_total*100:.0f}%)")
except:
    pass

CuPy память очищена

--- СОСТОЯНИЕ ПАМЯТИ ---
RAM: 8.7 / 13.6 GB (66.3%)
GPU: 0.2 / 15.0 GB (1%)


ПОДГОТОВКА ТЕСТОВЫХ ДАННЫХ И ПРЕДСКАЗАНИЯ

In [11]:
print("\nЗагрузка тестовых данных...")

candidates_df = pd.read_csv('/content/public/candidates.csv')
target_users_df = pd.read_csv('/content/public/targets.csv')

# Разворачиваем список книг-кандидатов в отдельные строки
candidates_df['book_id_list'] = candidates_df['book_id_list'].str.split(',')
test_pairs = candidates_df.explode('book_id_list').rename(
    columns={'book_id_list': 'book_id'}
).reset_index(drop=True)
test_pairs['book_id'] = test_pairs['book_id'].astype(int)

print(f"Пар user-book для предсказания: {len(test_pairs):,}")

# Генерируем признаки для тестовых данных
print("Генерация признаков для теста...")
test_features_df, _ = generate_features(
    test_pairs[['user_id', 'book_id']],
    interactions_df,
    members_df,
    items_df,
    latest_timestamp,
    member_embeddings=member_als_embeddings,
    item_embeddings=item_als_embeddings,
    writer_encoding=writer_te,
    category_encoding=category_te,
    recency_data=item_recency
)

# Подготавливаем признаки для предсказания
test_feature_matrix = test_features_df[feature_names].copy()
for col in feature_names:
    if col not in test_feature_matrix.columns:
        test_feature_matrix[col] = 0.0
test_feature_matrix = test_feature_matrix.fillna(0)

# Конвертируем категориальные признаки
for col in categorical_features:
    if col in test_feature_matrix.columns:
        test_feature_matrix[col] = test_feature_matrix[col].astype('category')


Загрузка тестовых данных...
Пар user-book для предсказания: 81,048
Генерация признаков для теста...


ПРЕДСКАЗАНИЯ И ФОРМИРОВАНИЕ SUBMISSION

In [12]:
print("Предсказания ансамблем...")

# Усредняем предсказания всех моделей
ensemble_predictions = np.zeros(len(test_feature_matrix), dtype=np.float64)

for model in trained_models:
    test_pool = Pool(
        data=test_feature_matrix,
        group_id=test_features_df['user_id'].astype(str),
        cat_features=categorical_features if categorical_features else None
    )
    ensemble_predictions += model.predict(test_pool)

ensemble_predictions /= len(trained_models)

# Добавляем скоры к парам user-book
test_pairs['score'] = ensemble_predictions

# Ранжируем книги для каждого пользователя
print("Формирование submission...")

ranked_results = test_pairs.sort_values(
    by=['user_id', 'score'],
    ascending=[True, False]
)

# Берём топ-20 книг для каждого пользователя
submission_grouped = (
    ranked_results
    .groupby('user_id')['book_id']
    .apply(lambda x: ','.join(x.astype(str).head(20)))
    .reset_index()
    .rename(columns={'book_id': 'book_id_list'})
)

# Объединяем с целевыми пользователями
final_submission = target_users_df.merge(submission_grouped, on='user_id', how='left')
final_submission['book_id_list'] = final_submission['book_id_list'].fillna('')

# Сохраняем результат
final_submission.to_csv('submission.csv', index=False)

print("\n" + "=" * 50)
print("ГОТОВО!")
print("=" * 50)
print(f"Submission сохранён: submission.csv")
print(f"Пользователей: {len(final_submission)}")
print("\nПервые строки submission:")
print(final_submission.head())

Предсказания ансамблем...
Формирование submission...

ГОТОВО!
Submission сохранён: submission.csv
Пользователей: 3512

Первые строки submission:
   user_id                                       book_id_list
0      210  3015694,1673950,2447113,3988468,971259,1281035...
1     1380  482934,2548861,2186305,8467358,2356900,2209477...
2     2050  2053462,2254200,1918727,2300795,1240867,317849...
3     2740  162418,1737221,1553798,181062,112023,2107128,1...
4     4621  3015694,2191492,28901,1838666,28638,2576738,31...


In [15]:
# ============================================================================
# СОХРАНЕНИЕ И СКАЧИВАНИЕ МОДЕЛЕЙ CATBOOST
# ============================================================================

import os
from google.colab import files

# Папка для сохранения
MODELS_DIR = '/content/catboost_info'
os.makedirs(MODELS_DIR, exist_ok=True)

# Сохраняем каждую модель
for i, model in enumerate(trained_models):
    model_path = f'{MODELS_DIR}/model_{i}.cbm'
    model.save_model(model_path)
    print(f"Сохранена: {model_path}")

# Архивируем
import shutil
shutil.make_archive('catboost_models', 'zip', MODELS_DIR)
print("\nАрхив создан: catboost_models.zip")

# Скачиваем
files.download('catboost_models.zip')

Сохранена: /content/catboost_info/model_0.cbm
Сохранена: /content/catboost_info/model_1.cbm

Архив создан: catboost_models.zip


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>